In [1]:
# Importacion de librerias
# Tratamiento de datos
# ==========================================
import re
import time
import pandas as pd
import numpy as np
import data_describe as dd
from string import digits
from gensim.utils import simple_preprocess
from gensim.corpora.textcorpus import strip_multiple_whitespaces
# Graficas
# ==========================================
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
# API REST
# ==========================================
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

import requests

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Define the API endpoint URL
url = "https://www.datos.gov.co/resource/f789-7hwg.json?nombre_entidad=INSTITUTO NACIONAL PENITENCIARIO Y CARCELARIO INPEC&$limit=100000"

# Send an HTTP GET request to the API endpoint
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    print("Data received from API:")
else:
    print(f"Request failed with status code {response.status_code}")


Data received from API:


In [3]:
pd.options.display.max_columns = None
# Extract API
secop_I = pd.DataFrame(data=data)

print(f"Filas:{secop_I.shape[0]}\nColumnas:{secop_I.shape[1]}")
secop_I.head(2)

Filas:64782
Columnas:69


,uid,anno_cargue_secop,anno_firma_contrato,nivel_entidad,orden_entidad,nombre_entidad,nit_de_la_entidad,c_digo_de_la_entidad,id_modalidad,modalidad_de_contratacion,estado_del_proceso,causal_de_otras_formas_de,id_regimen_de_contratacion,nombre_regimen_de_contratacion,id_objeto_a_contratar,objeto_a_contratar,detalle_del_objeto_a_contratar,tipo_de_contrato,municipio_de_obtencion,municipio_de_entrega,municipios_ejecucion,fecha_de_cargue_en_el_secop,numero_de_constancia,numero_de_proceso,numero_de_contrato,cuantia_proceso,id_grupo,nombre_grupo,id_familia,nombre_familia,id_clase,nombre_clase,id_adjudicacion,tipo_identifi_del_contratista,identificacion_del_contratista,nom_razon_social_contratista,dpto_y_muni_contratista,tipo_doc_representante_legal,identific_representante_legal,nombre_del_represen_legal,plazo_de_ejec_del_contrato,rango_de_ejec_del_contrato,tiempo_adiciones_en_dias,tiempo_adiciones_en_meses,compromiso_presupuestal,cuantia_contrato,valor_total_de_adiciones,valor_contrato_con_adiciones,objeto_del_contrato_a_la,proponentes_seleccionados,calificacion_definitiva,id_sub_unidad_ejecutora,nombre_sub_unidad_ejecutora,ruta_proceso_en_secop_i,moneda,es_postconflicto,marcacion_adiciones,posicion_rubro,nombre_rubro,valor_rubro,sexo_replegal,pilar_acuerdo_paz,punto_acuerdo_paz,municipio_entidad,departamento_entidad,ultima_actualizacion,fecha_liquidacion,cumpledecreto248,incluyebienesdecreto248
0,12-13-992986,2012,Sin Firma,NACIONAL,NACIONAL DESCENTRALIZADO,INSTITUTO NACIONAL PENITENCIARIO Y CARCELARIO ...,800215546,112008000,13,Contratación Mínima Cuantía,Liquidado,No Definido,13,Contratación Mínima Cuantía,31000000,Componentes y Suministros de Manufactura,Adquisición de elementos y materiales de const...,Suministro,Bogotá D.C.,Bogotá D.C.,"Bogotá D.C., Bogotá D.C.",2012-06-19T00:00:00.000,12-13-992986,113-SMC-0198-2012,No definido,1319000,D,[D] Componentes y Suministros,N/D,No Definido,N/D,No Definido,Sin Adjudicar,No Definido,No Definido,No Definido,No Definido,No Definido,No Definido,No Definido,0,N,0,0,Sn Definir,0,0,0,No Definido,No Definido,No Definido,0,No Definido,{'url': 'https://www.contratos.gov.co/consulta...,No Definido,0,0,No Definido,No Definido,0,N,No Definido,No Definido,Bogotá DC,Bogotá DC,2012-06-27T00:00:00.000,2014-10-07T00:00:00.000,N/D,N/D
1,17-13-6834143,2017,Sin Firma,NACIONAL,NACIONAL DESCENTRALIZADO,INSTITUTO NACIONAL PENITENCIARIO Y CARCELARIO ...,800215546,112008000,13,Contratación Mínima Cuantía,Celebrado,No Definido,13,Contratación Mínima Cuantía,50000000,"Alimentos, Bebidas y Tabaco",CONTRATAR EL SUMINISTRO DE MATERIA PRIMA E INS...,Suministro,Yopal,Yopal,"Yopal, Casanare",2017-07-19T00:00:00.000,17-13-6834143,EPC.YOP-MC-036-2017,No definido,23796998,E,[E] Productos de Uso Final,5018,Productos de panadería,501817,Suministros y mezclas para hornear,Sin Adjudicar,No Definido,No Definido,No Definido,No Definido,No Definido,No Definido,No Definido,0,N,0,0,Sn Definir,0,0,0,No Definido,No Definido,No Definido,0,No Definido,{'url': 'https://www.contratos.gov.co/consulta...,No Definido,0,0,No Definido,No Definido,0,N,No Definido,No Definido,Bogotá DC,Bogotá DC,2017-07-25T00:00:00.000,NaN,N/D,N/D


In [4]:
tabla_modalidad = secop_I.groupby(['id_objeto_a_contratar', 'objeto_a_contratar']).size().reset_index(name='cantidad')
tabla_modalidad

,id_objeto_a_contratar,objeto_a_contratar,cantidad
0,10000000,"Material Vivo Vegetal y Animal, Accesorios y S...",3386
1,11000000,"Material Mineral, Textil y Vegetal y Animal N...",901
2,12000000,Material Químico incluyendo Bioquímicos y Mate...,463
3,13000000,"Materiales de Resina, Colofonia, Caucho, Espum...",359
4,14000000,Materiales y Productos de Papel,1856
5,15000000,"Materiales Combustibles, Aditivos para Combust...",2677
6,20000000,Maquinaria y Accesorios de Minería y Perforaci...,16
7,21000000,"Maquinaria y Accesorios para Agricultura, Pesc...",232
8,22000000,Maquinaria y Accesorios para Construcción y Ed...,357
9,23000000,Maquinaria y Accesorios para Manufactura y Pro...,597


#### Filtramos data 

1. Filtramos por la columna `estado_del_proceso` los contratos que no se ejecutaron en su finalidad.

In [87]:
pd.DataFrame(secop_I['estado_del_proceso'].value_counts())

,estado_del_proceso
Celebrado,31927
Liquidado,19242
Terminado Anormalmente después de Convocado,5938
Convocado,5416
Terminado sin Liquidar,1559
Borrador,342
Adjudicado,195
Descartado,161
Expresión de Interés,2


Filtramos los contratos que se indice el valor de `Terminado Anormalmente después de Convocado`	y `Expresión de Interés`

In [6]:
anos =["Terminado Anormalmente después de Convocado", "Expresión de Interés"]
secop_I_filtro_estado = secop_I[~secop_I['estado_del_proceso'].isin(anos)]
print(f"Filas:{secop_I_filtro_estado.shape[0]}\nColumnas:{secop_I_filtro_estado.shape[1]}")
secop_I_filtro_estado.head(3)

Filas:58842
Columnas:69


,uid,anno_cargue_secop,anno_firma_contrato,nivel_entidad,orden_entidad,nombre_entidad,nit_de_la_entidad,c_digo_de_la_entidad,id_modalidad,modalidad_de_contratacion,estado_del_proceso,causal_de_otras_formas_de,id_regimen_de_contratacion,nombre_regimen_de_contratacion,id_objeto_a_contratar,objeto_a_contratar,detalle_del_objeto_a_contratar,tipo_de_contrato,municipio_de_obtencion,municipio_de_entrega,municipios_ejecucion,fecha_de_cargue_en_el_secop,numero_de_constancia,numero_de_proceso,numero_de_contrato,cuantia_proceso,id_grupo,nombre_grupo,id_familia,nombre_familia,id_clase,nombre_clase,id_adjudicacion,tipo_identifi_del_contratista,identificacion_del_contratista,nom_razon_social_contratista,dpto_y_muni_contratista,tipo_doc_representante_legal,identific_representante_legal,nombre_del_represen_legal,plazo_de_ejec_del_contrato,rango_de_ejec_del_contrato,tiempo_adiciones_en_dias,tiempo_adiciones_en_meses,compromiso_presupuestal,cuantia_contrato,valor_total_de_adiciones,valor_contrato_con_adiciones,objeto_del_contrato_a_la,proponentes_seleccionados,calificacion_definitiva,id_sub_unidad_ejecutora,nombre_sub_unidad_ejecutora,ruta_proceso_en_secop_i,moneda,es_postconflicto,marcacion_adiciones,posicion_rubro,nombre_rubro,valor_rubro,sexo_replegal,pilar_acuerdo_paz,punto_acuerdo_paz,municipio_entidad,departamento_entidad,ultima_actualizacion,fecha_liquidacion,cumpledecreto248,incluyebienesdecreto248
0,12-13-992986,2012,Sin Firma,NACIONAL,NACIONAL DESCENTRALIZADO,INSTITUTO NACIONAL PENITENCIARIO Y CARCELARIO ...,800215546,112008000,13,Contratación Mínima Cuantía,Liquidado,No Definido,13,Contratación Mínima Cuantía,31000000,Componentes y Suministros de Manufactura,Adquisición de elementos y materiales de const...,Suministro,Bogotá D.C.,Bogotá D.C.,"Bogotá D.C., Bogotá D.C.",2012-06-19T00:00:00.000,12-13-992986,113-SMC-0198-2012,No definido,1319000,D,[D] Componentes y Suministros,N/D,No Definido,N/D,No Definido,Sin Adjudicar,No Definido,No Definido,No Definido,No Definido,No Definido,No Definido,No Definido,0,N,0,0,Sn Definir,0,0,0,No Definido,No Definido,No Definido,0,No Definido,{'url': 'https://www.contratos.gov.co/consulta...,No Definido,0,0,No Definido,No Definido,0,N,No Definido,No Definido,Bogotá DC,Bogotá DC,2012-06-27T00:00:00.000,2014-10-07T00:00:00.000,N/D,N/D
1,17-13-6834143,2017,Sin Firma,NACIONAL,NACIONAL DESCENTRALIZADO,INSTITUTO NACIONAL PENITENCIARIO Y CARCELARIO ...,800215546,112008000,13,Contratación Mínima Cuantía,Celebrado,No Definido,13,Contratación Mínima Cuantía,50000000,"Alimentos, Bebidas y Tabaco",CONTRATAR EL SUMINISTRO DE MATERIA PRIMA E INS...,Suministro,Yopal,Yopal,"Yopal, Casanare",2017-07-19T00:00:00.000,17-13-6834143,EPC.YOP-MC-036-2017,No definido,23796998,E,[E] Productos de Uso Final,5018,Productos de panadería,501817,Suministros y mezclas para hornear,Sin Adjudicar,No Definido,No Definido,No Definido,No Definido,No Definido,No Definido,No Definido,0,N,0,0,Sn Definir,0,0,0,No Definido,No Definido,No Definido,0,No Definido,{'url': 'https://www.contratos.gov.co/consulta...,No Definido,0,0,No Definido,No Definido,0,N,No Definido,No Definido,Bogotá DC,Bogotá DC,2017-07-25T00:00:00.000,NaN,N/D,N/D
2,15-13-4478161,2015,Sin Firma,NACIONAL,NACIONAL DESCENTRALIZADO,INSTITUTO NACIONAL PENITENCIARIO Y CARCELARIO ...,800215546,112008000,13,Contratación Mínima Cuantía,Convocado,No Definido,13,Contratación Mínima Cuantía,47000000,Equipos y Suministros para Limpieza,COMPRA DE ELEMENTOS DE SEGURIDAD INDUSTRIAL,Suministro,Santander de Quilichao,Santander de Quilichao,"Santander de Quilichao, Cauca",2015-12-09T00:00:00.000,15-13-4478161,207-SMC-66-2015,No definido,250800,E,[E] Productos de Uso Final,4712,Equipo de aseo,471218,Equipo de limpieza,Sin Adjudicar,No Definido,No Definido,No Definido,No Definido,No Definido,No Definido,No Definido,0,N,0,0,Sn Definir,0,0,0,No Definido,No Definido,No Definido,0,No Definido,{'url': 'https://www.contratos.gov.co/consulta...,No Definido,0,0,No Definido,No Definido,0,N,No Definido,No D

#### Tipo de cada columna

A continuación podemos observar cada tipo de columna y la memoria que utiliza el proceso.

In [7]:
# Tipo de cada columna
# ==============================================================================
# En pandas, el tipo "object" hace referencia a strings
# datos.dtypes
secop_I_filtro_estado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58842 entries, 0 to 64781
Data columns (total 69 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   uid                             58842 non-null  object
 1   anno_cargue_secop               58842 non-null  object
 2   anno_firma_contrato             58842 non-null  object
 3   nivel_entidad                   58842 non-null  object
 4   orden_entidad                   58842 non-null  object
 5   nombre_entidad                  58842 non-null  object
 6   nit_de_la_entidad               58842 non-null  object
 7   c_digo_de_la_entidad            58842 non-null  object
 8   id_modalidad                    58842 non-null  object
 9   modalidad_de_contratacion       58842 non-null  object
 10  estado_del_proceso              58842 non-null  object
 11  causal_de_otras_formas_de       58842 non-null  object
 12  id_regimen_de_contratacion      58842 non-null

#### Número de observaciones y valores ausentes

Junto con el estudio del tipo de variables, es básico conocer el número de observaciones disponibles y si todas ellas están completas. Los valores ausentes son muy importantes a la hora de crear modelos, la mayoría de algoritmos no aceptan observaciones incompletas o bien se ven muy influenciados por ellas.

In [9]:
# Dimensiones del dataset
# ==============================================================================
print("Número de filas:", secop_I_filtro_estado.shape[0])
print("Número de columnas:", secop_I_filtro_estado.shape[1])

Número de filas: 58842
Número de columnas: 69


In [11]:
#pd.set_option('display.max_rows', None)
# Número de datos ausentes por variable
# ==============================================================================
pd.DataFrame(secop_I_filtro_estado.isna().sum().sort_values(ascending=False), columns=["Cantidad de Valores Nulos"]).head(13)

,Cantidad de Valores Nulos
fecha_liquidacion,39654
calificacion_definitiva,126
proponentes_seleccionados,39
tiempo_adiciones_en_meses,0
objeto_del_contrato_a_la,0
valor_contrato_con_adiciones,0
valor_total_de_adiciones,0
cuantia_contrato,0
compromiso_presupuestal,0
uid,0


#### Remover columnas innecesarias

Para tener un mejor panorama de los datos se realizo una entrevista a un contratista vigente del estado (Mario Rivera), el cual asesoro acerca de contratación publica y determino las variables mas importantes a tener en cuenta para la detección de patrones de contratación, adiciones y ejecuciones que ayude con el objetivo de  ofrecer información de valor al INPEC para la toma de decisiones que impacte positivamente en la administración de su contratación pública.

A continuación se eliminaran las columnas que no sean representativas para el requerimiento del cliente.

In [60]:
secop_I_filter = secop_I_filtro_estado[['anno_cargue_secop','id_modalidad', 'modalidad_de_contratacion',
                          'estado_del_proceso', 'id_objeto_a_contratar',
                          'detalle_del_objeto_a_contratar', 'tipo_de_contrato', 'municipios_ejecucion',
                          'cuantia_proceso','id_grupo', 'nombre_grupo','tipo_identifi_del_contratista', 'identificacion_del_contratista',
                          'plazo_de_ejec_del_contrato', 'rango_de_ejec_del_contrato', 'tiempo_adiciones_en_dias','tiempo_adiciones_en_meses',
                          'cuantia_contrato', 'valor_total_de_adiciones', 'valor_contrato_con_adiciones'
                          ]]
secop_I_filter.reset_index(drop=True, inplace=True)
secop_I_filter.head(2)

,anno_cargue_secop,id_modalidad,modalidad_de_contratacion,estado_del_proceso,id_objeto_a_contratar,detalle_del_objeto_a_contratar,tipo_de_contrato,municipios_ejecucion,cuantia_proceso,id_grupo,nombre_grupo,tipo_identifi_del_contratista,identificacion_del_contratista,plazo_de_ejec_del_contrato,rango_de_ejec_del_contrato,tiempo_adiciones_en_dias,tiempo_adiciones_en_meses,cuantia_contrato,valor_total_de_adiciones,valor_contrato_con_adiciones
0,2012,13,Contratación Mínima Cuantía,Liquidado,31000000,Adquisición de elementos y materiales de const...,Suministro,"Bogotá D.C., Bogotá D.C.",1319000,D,[D] Componentes y Suministros,No Definido,No Definido,0,N,0,0,0,0,0
1,2017,13,Contratación Mínima Cuantía,Celebrado,50000000,CONTRATAR EL SUMINISTRO DE MATERIA PRIMA E INS...,Suministro,"Yopal, Casanare",23796998,E,[E] Productos de Uso Final,No Definido,No Definido,0,N,0,0,0,0,0


### Normalización de los datos

* Se Separo **Municipios** y **Departamento** de una misma columna con la finalidad de segementar optimamente los datos en dos diferentes columnas

In [61]:
# Separo municipio y departamento
split_columns = secop_I_filter['municipios_ejecucion'].str.split(r'[;,]', expand=True)
data_mun_dep = split_columns.loc[:,:1]
data_mun_dep.columns = ['municipios_ejecucion_', 'departamento_ejecucion']
secop_I_filter['municipios_ejecucion_'] = data_mun_dep.municipios_ejecucion_.to_list()
secop_I_filter['departamento_ejecucion'] = data_mun_dep.departamento_ejecucion.to_list()

# Eliminar la columna original si ya no es necesaria
secop_I_filter.drop('municipios_ejecucion', axis=1, inplace=True)
# organizo las columhns de municipio o departamento
secop_I_filter = secop_I_filter[['anno_cargue_secop', 'id_modalidad', 'modalidad_de_contratacion',
       'estado_del_proceso', 'id_objeto_a_contratar', 'municipios_ejecucion_', 'departamento_ejecucion',
       'detalle_del_objeto_a_contratar', 'tipo_de_contrato', 'cuantia_proceso',
       'id_grupo', 'nombre_grupo', 'tipo_identifi_del_contratista',
       'identificacion_del_contratista', 'plazo_de_ejec_del_contrato',
       'rango_de_ejec_del_contrato', 'tiempo_adiciones_en_dias',
       'tiempo_adiciones_en_meses', 'cuantia_contrato',
       'valor_total_de_adiciones', 'valor_contrato_con_adiciones',
       ]]
pd.DataFrame(secop_I_filter.departamento_ejecucion.value_counts())

,departamento_ejecucion
Antioquia,7455
Boyacá,4815
Bogotá D.C.,4632
Santander,4625
Valle del Cauca,4146
Cundinamarca,3923
Tolima,3665
Meta,3617
Caldas,3489
Cauca,3120


*  Para las columnas de **plazo_de_ejec_del_contrato** y **rango_de_ejec_del_contrato** se normaliza en un tiempo de días unicamente. 
Dado que la duración real de un mes puede variar, se utiliza la duración de 30 días.

In [56]:
# Función para convertir el dataframe 
def convert_months_to_days(row):
    if row['rango_de_ejec_del_contrato'] == "M":
        average_days_per_month = 30 #30.44
        days = int(row['plazo_de_ejec_del_contrato']) * average_days_per_month
        return days
    return row['plazo_de_ejec_del_contrato']

In [57]:
# Convierto la columna en tipo INTEGER
secop_I_filter['plazo_de_ejec_del_contrato'] = secop_I_filter['plazo_de_ejec_del_contrato'].astype(int)
secop_I_filter['plazo_de_ejec_del_contrato_'] = secop_I_filter.apply(convert_months_to_days, axis=1)

# Remplazo rango_de_ejec_del_contrato por dias en todas las filas
secop_I_filter['rango_de_ejec_del_contrato'] = secop_I_filter['rango_de_ejec_del_contrato'].replace('M', 'D')

In [62]:
secop_I_filter.rango_de_ejec_del_contrato.value_counts()

N    58842
Name: rango_de_ejec_del_contrato, dtype: int64

In [63]:
secop_I_filter.plazo_de_ejec_del_contrato.value_counts()

0    58842
Name: plazo_de_ejec_del_contrato, dtype: int64

In [66]:
secop_I_filter.tiempo_adiciones_en_dias.value_counts()

0    58842
Name: tiempo_adiciones_en_dias, dtype: int64

In [83]:
secop_I_filter.cuantia_contrato.value_counts()

0    58842
Name: cuantia_contrato, dtype: int64

Se puede observar que los datos cambiaron y las columnas de `rango_de_ejec_del_contrato`, `plazo_de_ejec_del_contrato` y `tiempo_adiciones_en_dias` aparen en 0 en todos los contratos de todos los años. Esto nos indica que hubo una modificación o manipulación de los datos de origen que produce un sesgo en la significancia de estas columnas y la importancia que tenian en el analisis de nuestro proyecto.

#### Se eliminan las columnas

In [74]:
secop_I_filter.drop(['plazo_de_ejec_del_contrato','tiempo_adiciones_en_dias','rango_de_ejec_del_contrato'], axis=1, inplace=True)

### Data incompleta - `No Definida`

Por medio de tecnica scramble html se puede acceder a la Hoja del proceso de cada contrato con la finalidad de obtener información faltante de la base de datos y asi poder completar información de valor para el analisis.

Esta base de datos define estos datos faltante como valore ``No Definido``, la idea es rellenar estos valores con los extraidos de la tecnica de scramble

In [67]:
def found_miss_data(dataframe:pd.DataFrame, valor_especifico:str):
    # Establecer el valor específico que deseas encontrar
    valor_especifico = valor_especifico
    # Utilizar el método 'isin()' para encontrar las columnas que contienen el valor específico
    columnas_con_valor_especifico = dataframe.isin([valor_especifico]).any()
    # Filtrar las columnas que contienen el valor específico
    columnas_filtradas = columnas_con_valor_especifico[columnas_con_valor_especifico].index.tolist()
    # Contar la cantidad de veces que se repite el valor específico en cada columna
    specific_value_counts = pd.DataFrame(dataframe.apply(lambda x: (x == valor_especifico).sum()), columns=["count"])

    return specific_value_counts, columnas_filtradas

In [68]:
specific_value_counts, columnas_filtradas = found_miss_data(secop_I_filter, 'No Definido')
print("\nNúmero de columnas con valores [No Definido]:",len(columnas_filtradas))
specific_value_counts[specific_value_counts["count"] != 0].sort_values(by=['count'])


Número de columnas con valores [No Definido]: 2


,count
tipo_identifi_del_contratista,58842
identificacion_del_contratista,58842


In [70]:
specific_value_counts, columnas_filtradas = found_miss_data(secop_I_filter, 'no definido')
print("\nNúmero de columnas con valores [No Definido]:",len(columnas_filtradas))
specific_value_counts[specific_value_counts["count"] != 0].sort_values(by=['count'])


Número de columnas con valores [No Definido]: 0


,count


Como observar las columnas `tipo_identifi_del_contratista` y `identificacion_del_contratista` todos sus valores son No definidos, es decir, no existe ninguna información de valor que nos aporte al modelo o analisis.

### Data incompleta - `N/D`

Por medio de tecnica scramble html se puede acceder a la Hoja del proceso de cada contrato con la finalidad de obtener información faltante de la base de datos y asi poder completar información de valor para el analisis.

Esta base de datos define estos datos faltante como valore ``N/D``, la idea es rellenar estos valores con los extraidos de la tecnica de scramble

In [69]:
specific_value_counts, columnas_filtradas = found_miss_data(secop_I_filter, 'N/D')
print("\nNúmero de columnas con valores [No Definido]:",len(columnas_filtradas))
specific_value_counts[specific_value_counts["count"] != 0].sort_values(by=['count'])


Número de columnas con valores [No Definido]: 0


,count


#### Definición DTYPES

Se define un tipo de dato para cada columna

In [75]:
secop_I_filter.dtypes

anno_cargue_secop                  int64
id_modalidad                       int64
modalidad_de_contratacion         object
estado_del_proceso                object
id_objeto_a_contratar              int64
municipios_ejecucion_             object
departamento_ejecucion            object
detalle_del_objeto_a_contratar    object
tipo_de_contrato                  object
cuantia_proceso                   object
id_grupo                          object
nombre_grupo                      object
tipo_identifi_del_contratista     object
identificacion_del_contratista    object
tiempo_adiciones_en_meses         object
cuantia_contrato                  object
valor_total_de_adiciones          object
valor_contrato_con_adiciones      object
dtype: object

In [77]:
secop_I_filter.cuantia_contrato

0        0
1        0
2        0
3        0
4        0
        ..
58837    0
58838    0
58839    0
58840    0
58841    0
Name: cuantia_contrato, Length: 58842, dtype: object

In [73]:
secop_I_filter[['anno_cargue_secop', 'id_modalidad','id_objeto_a_contratar']] = secop_I_filter[['anno_cargue_secop', 'id_modalidad', 'id_objeto_a_contratar']].astype(int)
secop_I_filter[['cuantia_contrato','plazo_de_ejec_del_contrato_']] = secop_I_filter[['cuantia_contrato','plazo_de_ejec_del_contrato_']].astype(int)
secop_I_filter[['valor_total_de_adiciones','valor_contrato_con_adiciones']] = secop_I_filter[['valor_total_de_adiciones','valor_contrato_con_adiciones']].astype(int)

KeyError: "['plazo_de_ejec_del_contrato_'] not in index"